# Methodology
We will perform a two-tail, two-sample t-test on the data. Using [Kaggle's UW Madison Grade's Database](https://www.kaggle.com/Madgrades/uw-madison-courses#schedules.csv).

## Question

Is a statitstically significant difference in the percentage of A's based on how many days a week a class meets.

## Hypotheses
We select an alpha value of 0.05.

Null Hypothesis: There is no significant difference in the mean percentage of A's by the number of days a class meets.

Alternate Hypothesis: There is a significant difference in the mean percentage of A's  by the number of days a class meets.

Construct an ANOVA test.


In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
conn = psycopg2.connect("host=localhost,dbname=postgres,user=postgres")

cur = conn.cursor()

In [3]:
group_and_as = """
SELECT 
  grade.a_count
, grade.ab_count
, grade.b_count
, grade.bc_count
, grade.c_count
, grade.d_count
, grade.f_count
, sched.mon
, sched.tues
, sched.wed
, sched.thurs
, sched.fri
, sched.sat
, sched.sun
, sect.section_type
, grade.course_offering_uuid
, grade.section_number
FROM 
  sections sect
INNER JOIN 
  schedules sched 
ON 
  sched.uuid = sect.schedule_uuid
INNER JOIN 
  grade_distributions grade 
ON 
  sect.number = grade.section_number 
AND 
  sect.course_offering_uuid = grade.course_offering_uuid;
"""

In [4]:
cur.execute(group_and_as)
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]

In [5]:
df['total_count'] = (df['a_count'] + df['ab_count'] + df['b_count'] + df['bc_count'] 
                     + df['c_count'] + df['d_count'] + df['f_count'])

In [6]:
df['days_a_week'] = sum([df['mon'], df['tues'], df['wed'], df['thurs'], df['fri'], df['sat'], df['sun']])

In [7]:
df.head()

,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,mon,tues,wed,thurs,fri,sat,sun,section_type,course_offering_uuid,section_number,total_count,days_a_week
0,158,0,0,0,0,0,0,False,False,False,False,False,False,False,FLD,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0
1,0,0,0,0,0,0,0,False,False,False,False,False,False,False,FLD,36ab1e06-94ef-363f-841c-0e923bfbdac7,10,0,0
2,0,0,0,0,0,0,0,False,False,False,False,False,False,False,FLD,aeef8e65-6b2c-3c7a-aeba-3f594b9255df,35,0,0
3,0,0,0,0,0,0,0,False,False,False,False,False,False,False,FLD,94d362f0-53b2-33d0-85ca-4770d1371c97,20,0,0
4,0,0,0,0,0,0,0,False,False,False,False,False,False,False,FLD,94d362f0-53b2-33d0-85ca-4770d1371c97,33,0,0


In [8]:
df['days_a_week'].value_counts()

0    92413
2    48620
1    42123
3    17703
6     7556
5     5150
4     3811
7       23
Name: days_a_week, dtype: int64

In [9]:
df.sort_values(by='days_a_week', ascending=False).head()

,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,mon,tues,wed,thurs,fri,sat,sun,section_type,course_offering_uuid,section_number,total_count,days_a_week
167904,1,3,15,5,0,0,0,True,True,True,True,True,True,True,FLD,6aec025e-376b-3119-b32d-8b64abfd5c10,81,24,7
4887,4,10,12,0,0,0,0,True,True,True,True,True,True,True,FLD,032e8a6c-880a-3f71-ac52-e073091dff1d,82,26,7
95083,8,6,19,0,0,0,0,True,True,True,True,True,True,True,FLD,42166827-1020-3daa-803f-30b5a0598d5c,82,33,7
77038,6,9,12,0,0,0,0,True,True,True,True,True,True,True,FLD,dea21d58-b5bb-331d-9a15-6de8b5fbc240,82,27,7
212072,3,12,9,0,0,0,0,True,True,True,True,True,True,True,FLD,d00c9390-ad72-3c64-8ed1-3195e2690095,81,24,7


In [10]:
df.sort_values(by='days_a_week', ascending=True).head()

,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,mon,tues,wed,thurs,fri,sat,sun,section_type,course_offering_uuid,section_number,total_count,days_a_week
0,158,0,0,0,0,0,0,False,False,False,False,False,False,False,FLD,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0
151419,0,0,0,0,0,0,0,False,False,False,False,False,False,False,IND,144c033f-5aef-3d8c-ab01-fffdca6e2372,17,0,0
151420,0,0,0,0,0,0,0,False,False,False,False,False,False,False,IND,93e34a5b-561a-3013-9b4d-e4e11d21b506,20,0,0
151421,0,0,0,0,0,0,0,False,False,False,False,False,False,False,IND,93e34a5b-561a-3013-9b4d-e4e11d21b506,3,0,0
151422,0,0,0,0,0,0,0,False,False,False,False,False,False,False,IND,93e34a5b-561a-3013-9b4d-e4e11d21b506,15,0,0


We can see that there are several classes that meet zero times a week. We want to drop these. We also see that there are classes that meet 6-7 days a week which are FLD or IND, which are not representative of our problem as these are usually student paced/scheduled. As all these values are no use to our test, we will drop classes that meet 0, 6, or 7 days a week.

In [11]:
df = df[df['days_a_week'] < 6]
df = df[df['days_a_week'] > 0]
df.sort_values(by='days_a_week', ascending=False).head()

,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,mon,tues,wed,thurs,fri,sat,sun,section_type,course_offering_uuid,section_number,total_count,days_a_week
92654,0,0,0,0,0,0,0,True,True,True,True,True,False,False,LEC,53fafe2f-8e38-3f55-8d51-899fccc985ff,4,0,5
185586,2,0,2,0,0,0,0,True,True,True,True,True,False,False,LEC,3ee6b2b9-9ad9-3a98-a9a6-a3a053ce3342,1,4,5
129930,11,0,0,0,0,0,0,True,True,True,True,True,False,False,FLD,0ea14948-7021-37b6-9807-ca9cb7bbb41c,1,11,5
129931,11,0,0,0,0,0,0,True,True,True,True,True,False,False,FLD,0ea14948-7021-37b6-9807-ca9cb7bbb41c,1,11,5
188569,17,0,0,0,0,0,0,True,True,True,True,True,False,False,LEC,2135a4d8-1d2d-3bc9-94a9-b216c66bb0c0,1,17,5


There are some lectures that meet 5 days a week. So we will drop any remaining FLD or IND.

In [12]:
df = df[df['section_type'] != 'FLD']
df = df[df['section_type'] != 'IND']
df.sort_values(by='days_a_week', ascending=False).head()

,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,mon,tues,wed,thurs,fri,sat,sun,section_type,course_offering_uuid,section_number,total_count,days_a_week
155371,0,0,0,0,0,0,0,True,True,True,True,True,False,False,LEC,eecbecfe-6e61-347f-a26f-0840c95770b7,1,0,5
26287,8,3,4,1,3,1,0,True,True,True,True,True,False,False,LEC,05ac54d2-820b-3fe0-bb96-d405c641eca1,3,20,5
176504,4,0,11,0,6,0,0,True,True,True,True,True,False,False,LEC,fcced232-7712-3360-bd6f-8ae31aeaa45d,1,21,5
16532,5,3,1,2,1,0,0,True,True,True,True,True,False,False,LEC,695b6f0f-ed48-347e-acdf-a2f9469e8a8c,1,12,5
99878,0,0,0,0,0,0,0,True,True,True,True,True,False,False,LEC,6950519f-4c90-344a-8e20-222a61984678,7,0,5


In [ ]:
df.info()

Like with time of day, there may be some classes with zero students. We want to drop these as it is not reflective of our study.

In [ ]:
df = df[df['total_count'] != 0]

In [ ]:
df.info()

In [ ]:
df2 = df.copy()

In [ ]:
df2['ratio_of_as'] = df['a_count'] / df['total_count']

In [ ]:
df2.info()

In [ ]:
df2.drop_duplicates(inplace=True)
df2.info()

In [ ]:
sns.jointplot('days_a_week', 'ratio_of_as',df2)

Create new columns for one-hot encoding for days a class meets a week in order to perform a one-way ANOVA test

In [ ]:
pd.get_dummies(df2, columns='days_a_week')
df2.head()

In [ ]:
morning_df.info()

In [ ]:
afternoon_df.info()

In [ ]:
sns.jointplot('start_time', 'ratio_of_as',morning_df)

In [ ]:
sns.jointplot('start_time', 'ratio_of_as',afternoon_df)

# Determine Sample Size

In [ ]:
moe = 0.1
z = 1.96 # z value for alpha 0.05
morn_var = np.var(morning_df['ratio_of_as'])
morn_n = len(morning_df['ratio_of_as'])
aft_var = np.var(afternoon_df['ratio_of_as'])
aft_n = len(afternoon_df['ratio_of_as'])

num = (morn_n - 1) * morn_var + (aft_n - 1) * aft_var
den = morn_n + aft_n - 2
pooled_std = np.sqrt(num/den)

In [ ]:
n = 2 * (z * pooled_std / moe)**2
n

We will use samples from morning and afternoon of size 68.

In [ ]:
morning = morning_df['ratio_of_as']
afternoon = afternoon_df['ratio_of_as']

morning_sample = np.random.choice(morning, size=68)
afternoon_sample = np.random.choice(afternoon, size=68)

In [ ]:
stats.ttest_ind(morning_sample, afternoon_sample)

Our p-value is about 0.127 which is greater than our alpha of 0.05. This means that we cannot reject the null hypothesis. We cannot say that there is a statistical difference in the ratio of A's of a class depending on if the class is before or after noon. 

If we want to improve our schedules to increase the percentage of A's we will need to look at other factors.